# Notebook tester

In [6]:
from IPython.core.magic import register_cell_magic
from IPython import get_ipython

@register_cell_magic
def caputre_png(line, cell):
    print("hello caputre_png")
    get_ipython().run_cell_magic(
        'capture',
        ' --no-stderr --no-stdout result',
        cell
    )
    for output in result.outputs:
        data = output.data
        print(data.keys())


In [7]:
%%caputre_png
import matplotlib.pyplot as plt
plt.plot([1,2],[10,20])

hello caputre_png
dict_keys(['text/plain'])
dict_keys(['text/plain', 'image/png'])


In [8]:
from my_tools import *

In [13]:
%%another_new_cellmagic
import matplotlib.pyplot as plt
plt.plot([1,2],[10,20])

Hello another_new_cellmagic


In [14]:
%%caputre_png_test
import matplotlib.pyplot as plt
plt.plot([1,2],[10,20])

hello caputre_png_test


NameError: name 'result' is not defined